In [ ]:
import hashlib
import re
import time
import datetime as dt
from itertools import chain
import requests
import logging
from bs4 import BeautifulSoup
from urllib.parse import urlencode
import csv

#from .signals import DictSIG
#from math import ceil

In [ ]:
cookie = {
    "DICT_LOGIN": "7||1551660703487",
    "DICT_SESS": "v2|iWMwhDxnqkMhfgLPMPu0l5kfzfn4Pu0lGP4lWPLU50gZ64lf0HwF0euRMgyRMpB0O50fYY0MQL0J4nMqLnHPBRkW0fkfnHPB0"
}

In [ ]:
def getTotalPage():
    try:
        rsp = requests.get('http://dict.youdao.com/wordbook/wordlist?tags=other', timeout=6000, cookies=cookie)
        groups = re.search('<a href="wordlist.p=(.*).tags=other" class="next-page">最后一页</a>', rsp.text, re.M | re.I)
        if groups:
            total = int(groups.group(1)) - 1
        else:
            total = 1
        return total
    except Exception as e:
        raise e

In [ ]:
def getWordPerPage(pageNumber):
    words = []
    try:
        print(f'获取单词本第:{pageNumber + 1}页')
        rsp = requests.get(
            'http://dict.youdao.com/wordbook/wordlist?tags=other',
            params={'p': pageNumber},
            cookies=cookie
        )
        soup = BeautifulSoup(rsp.text, features='html.parser')
        table = soup.find(id='wordlist').table.tbody
        rows = table.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            # word | date
            so = dt.datetime.strptime(cols[4].text.strip(), '%Y-%m-%d').timestamp()
            word_date = cols[1].div.a.text.strip() + '|' + str(so)
            words.append(word_date)
        self.SIG.progress.emit()
    except Exception as e:
        raise e
    finally:
        return words

In [ ]:
def parser(_result, term):
    
    # definitions
    try:
        ec = [d['tr'][0]['l']['i'][0] for d in _result['ec']['word'][0]['trs']][:3]
    except KeyError:
        ec = []

    try:
        web_trans = [w['value'] for w in _result['web_trans']['web-translation'][0]['trans']][:3]
    except KeyError:
        web_trans = []
    definitions = ec if ec else web_trans
    
    # pronuciations
    url = 'http://dict.youdao.com/dictvoice?audio='
    pron = {
        'us_phonetic': None,
        'us_url': None,
        'uk_phonetic': None,
        'uk_url': None
    }
    try:
        pron['us_phonetic'] = _result['simple']['word'][0]['usphone']
        pron['us_url'] = url + _result['simple']['word'][0]['usspeech']
        pron['uk_phonetic'] = _result['simple']['word'][0]['ukphone']
        pron['uk_url'] = url + _result['simple']['word'][0]['ukspeech']
    except KeyError:
        pass
    
    # samples
    
    try:
        sentences = [(s['sentence']) for s in _result['blng_sents_part']['sentence-pair']]
        samples = [(s['sentence'], s['sentence-translation'],) for s in _result['blng_sents_part']['sentence-pair']]
        
    except KeyError:
        sentences = []
        samples = []
        
    # images
    try:
        image = [i['image'] for i in _result['pic_dict']['pic']][0]
    except KeyError:
        image = None
    
    return {
        "term": term,
        "definitions": ' '.join(definitions),
        "image": image,
        "sentencesHtml" : '<ul>' + ''.join([f'<li>{e}</li>' for e in sentences]) + '</ul>',
        "samplesHtml":  '<ul>' + ''.join([f'<li>{e}<br>{c}</li>' for e, c in samples]) + '</ul>',
        "uk": pron["uk_phonetic"],
        "us": pron["us_phonetic"],
        "pron": f'[sound:ame_{term}.mp3]'
    }

In [ ]:
def query(word):
    s = requests.Session()
    url = 'https://dict.youdao.com/jsonapi'
    params = {"dicts": {"count": 99, "dicts": [["ec", "pic_dict"], ["web_trans"], ["fanyi"], ["blng_sents_part"]]}}

    try:
        arr = word.split('|')
        word = arr[0]
        rsp = s.get(
            url,
            params=urlencode(dict(params, **{'q': word})),
        )
        jsonResult = parser(rsp.json(), word)
        jsonResult['order'] = float(arr[1])
        return jsonResult
    except Exception as e:
        raise e        

In [ ]:
words_chain = chain(*[getWordPerPage(n) for n in range(getTotalPage())])
words = list(words_chain)
wds = []
proceed = 0
for word in words:
    result = query(word)
    wds.append(result)
    if (proceed % 20 == 0):
        print(str(proceed) + ' words has been deal')
        proceed = proceed + 1


In [ ]:
wds = sorted(wds, key = lambda i: i['order'])
with open('youdao-wordlist.csv', 'w', encoding='utf-8') as cfo:
    fns = ['term','definitions','uk','us','image','sentencesHtml','samplesHtml','pron','order']
    writer = csv.DictWriter(cfo, fieldnames=fns, lineterminator='\n')
    writer.writeheader()
    for row in wds:
        writer.writerow(row)
print("导出完成")